# IMPORTING REQUIRED LIBRARY

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

# READ DATASET AND PREPROCESSING

In [2]:
df = pd.read_csv('/content/drive/MyDrive/IBM/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
X = df.v2
Y = df.v1
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)
Y = Y.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [5]:
tokenizer = Tokenizer(num_words=2000, lower=True)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences, maxlen=200)

# CREATE MODEL AND ADD LAYERS

In [6]:
model = Sequential()

In [7]:
model.add(Embedding(2000, 50, input_length=200))
model.add(LSTM(64))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           100000    
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 146,337
Trainable params: 146,337
Non-trainable params: 0
_________________________________________________________________


# COMPILE THE MODEL

In [9]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

# TRAIN AND FIT THE MODEL

In [11]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
28/28 [==============================] - 9s 34ms/step - loss: 0.3342 - accuracy: 0.8712 - val_loss: 0.1483 - val_accuracy: 0.9731
Epoch 2/10
28/28 [==============================] - 0s 15ms/step - loss: 0.0878 - accuracy: 0.9798 - val_loss: 0.0544 - val_accuracy: 0.9832
Epoch 3/10
28/28 [==============================] - 0s 14ms/step - loss: 0.0452 - accuracy: 0.9868 - val_loss: 0.0493 - val_accuracy: 0.9854
Epoch 4/10
28/28 [==============================] - 0s 14ms/step - loss: 0.0292 - accuracy: 0.9916 - val_loss: 0.0520 - val_accuracy: 0.9854
Epoch 5/10
28/28 [==============================] - 0s 14ms/step - loss: 0.0220 - accuracy: 0.9944 - val_loss: 0.0506 - val_accuracy: 0.9843
Epoch 6/10
28/28 [==============================] - 0s 14ms/step - loss: 0.0139 - accuracy: 0.9961 - val_loss: 0.0578 - val_accuracy: 0.9832
Epoch 7/10
28/28 [==============================] - 0s 14ms/step - loss: 0.0109 - accuracy: 0.9969 - val_loss: 0.0659 - val_accuracy: 0.9843
Epoch 8/10
28

# SAVE THE MODEL

In [12]:
model.save("model.h5")

# PREPROCESSING THE DATASET

In [13]:
test_sequences = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(test_sequences, maxlen=200)

# TESTING THE MODEL

In [15]:
acc = model.evaluate(X_test, y_test)

35/35 [==============================] - 0s 7ms/step - loss: 0.1129 - accuracy: 0.9874


In [17]:
def predict(message):
    txt = tokenizer.texts_to_sequences(message)
    txt = sequence.pad_sequences(txt, maxlen=200)
    preds = model.predict(txt)
    if preds > 0.5:
        print("Spam")
    else:
        print("Not Spam")

In [18]:
predict(["URGENT! Your Mobile number has been awarded a <UKP>2000 prize GUARANTEED. Call 09061790125 from landline. Claim 3030. Valid 12hrs only 150ppm,,,"])

1/1 [==============================] - 0s 402ms/step
Spam


In [19]:
predict(["I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today."])

1/1 [==============================] - 0s 20ms/step
Not Spam
